In [17]:
class Pet_hos:
    def __init__(self, a):
        self.a = a
    def test (self, bb):
        print(bb)
        print(self.a)

In [18]:
a = Pet_hos('1')
b = a.test('aa')
b

aa
1


In [152]:
import pandas as pd
import chardet
import re
import requests
import json
import folium
from folium.plugins import MarkerCluster
import webbrowser
from bokeh.plotting import show
from bokeh.models import ColumnDataSource, DataTable, TableColumn
import bokeh



class Veterinary :

    '''
    동물병원 전처리
    csv 출처(행안부) https://www.data.go.kr/data/15045050/fileData.do
    '''

    def __init__(self, filename :str ="./fulldata_02_03_01_P_동물병원.csv") -> None :
        self.filename = filename

    def find_encoding(self) -> pd.DataFrame:
        read_file = open(self.filename, 'rb').read()        # 파일 읽기모드 rb는 read + 바이너리 파일( 바이너리 파일 읽을때 사용 )
        result    = chardet.detect(read_file)
        charenc   = result['encoding']
        open_source = pd.read_csv(source_path, encoding=charenc)
        # self.open_source1 = open_source.loc[((open_source['상세영업상태명']=="정상") | (open_source['상세영업상태명']=="휴업")),:]
        open_source1 = open_source.loc[open_source['상세영업상태명']=="정상",:]
        print(f'파일경로 : {self.filename}')
        print(f'파일 인코딩 방식 : {charenc}')
        return open_source1

    def choose_columns(self) -> pd.DataFrame:
        open_source = self.find_encoding()
        # c = []
        # while True:
        #     print(list(self.open_source1.columns))
        #     b = input("추가할 컬럼을 선택 // 종료 0")
        #     if b == '0':
        #         break
        #     c.append(b)
        # self.df = self.open_source1.loc[:,c]
        self.fin_source_data = open_source.iloc[:,[21,10,18,19,26,27]]
        self.fin_source_data['구분'] = self.fin_source_data['소재지전체주소'].str.split(' ').str.get(0).copy()
        self.fin_source_data.columns= ['병원명', '영업상태', '지번', '도로명', '위도', '경도', '지역']
        self.fin_source_data['도로명'].fillna(0, inplace=True)
        return self.fin_source_data

    def test_list(self) -> list[str]:
        self.choose_columns()
        addr_final = list()
        for i,j in self.fin_source_data[['도로명']].iterrows():
            if j[0] != 0:
                addr_final.append(j[0])
            else :
                self.fin_source_data.loc[i,"도로명"] = self.fin_source_data.loc[i,"지번"] #########
                addr_final.append(self.fin_source_data.loc[i,"지번"])
                # 비교대상 리스트
        
        return addr_final

    def test_list2(self) -> pd.DataFrame:
        #self.test_list()
        addr_final = self.test_list()
        count = 0
        exc = list()
        for i in addr_final :
            x_y = dict(위도='y', 경도='x')
            self.rest_api_key = 'c6ead90ce93e6ff182fbb5dbbb2efb6c'
            query = i
            url = f"https://dapi.kakao.com/v2/local/search/address.json?query={query}"
            headers={'Authorization':'KakaoAK {}'.format(self.rest_api_key)}
            res = requests.get(url, headers=headers)
            res_json = res.json()
            try :
                #print(f'POINT 1{i}')
                #print(f'POINT 2{self.fin_source_data.loc[ self.fin_source_data["도로명"]== i, "도로명" ].unique()[0]}')

                if i == self.fin_source_data.loc[ self.fin_source_data["도로명"]== i, "도로명" ].unique()[0]:
                    for j,k in x_y.items():
                        self.fin_source_data.loc[ self.fin_source_data["도로명"]== i , j ] = res_json['documents'][0]['address'][k]
                elif  i == self.fin_source_data.loc[ self.fin_source_data["지번"]== i, "지번" ].unique()[0]:
                    self.fin_source_data.loc[ self.fin_source_data["지번"]== i , "도로명" ] = res_json['documents'][0]['road_address']['address_name']
                    for j,k in x_y.items():
                            self.fin_source_data.loc[ self.fin_source_data["지번"]== i , j ] = res_json['documents'][0]['address'][k]
                        
                
                #addr = res_json['documents'][0]['road_address']['address_name']
                name = self.fin_source_data.loc[ self.fin_source_data["도로명"]== i, "병원명" ].unique()[0]
                #print("point 12 comp")
                count += 1
            except IndexError:
                try:

                    i2 = i.split()
                    i2 = ' '.join(i2[:4])
                    i2 = re.sub(r'[,]','',i2)
                    query = i2
                    url = f"https://dapi.kakao.com/v2/local/search/address.json?query={query}"
                    headers={'Authorization':'KakaoAK {}'.format(self.rest_api_key)}
                    res = requests.get(url, headers=headers)
                    res_json = res.json()

                    #print(f'POINT 3{i}')
                    #print(f'POINT 3.5{i2}')
                    #print(f'POINT 4{self.fin_source_data.loc[ self.fin_source_data["도로명"]== i, "도로명" ].unique()[0]}')

                    if i == self.fin_source_data.loc[ self.fin_source_data["도로명"]== i, "도로명" ].unique()[0]:
                        for j,k in x_y.items():
                            self.fin_source_data.loc[ self.fin_source_data["도로명"]== i , j ] = res_json['documents'][0]['address'][k]
                    elif  i == self.fin_source_data.loc[ self.fin_source_data["지번"]== i, "지번" ].unique()[0]:
                        self.fin_source_data.loc[ self.fin_source_data["지번"]== i , "도로명" ] = res_json['documents'][0]['road_address']['address_name']
                        for j,k in x_y.items():
                                self.fin_source_data.loc[ self.fin_source_data["지번"]== i , j ] = res_json['documents'][0]['address'][k]
                            
                    #addr = res_json['documents'][0]['road_address']['address_name']
                    name = self.fin_source_data.loc[ self.fin_source_data["도로명"]== i, "병원명" ].unique()[0]
                    count += 1
                except:
                    print("---- 오류 주소 내용----")
                    print(i)
                    exc.append(i)
        else:
            print(f'좌표 입력 횟수{count}')
            print(f'좌표 오류 횟수{len(exc)}')
            #print(f'좌표 오류 내용{exc}')
            return self.fin_source_data

    def make_map(self) -> folium.map:
        fin_source_data =  self.test_list2()
        m = folium.Map(location=[35.1358104681195,129.045358241731],zoom_start=8)
        marker_c = MarkerCluster().add_to(m)
        kerr = list()
        count = 0
        for _,j in fin_source_data.iterrows():
            try:
                x = j["경도"]
                y = j["위도"]
                url = f"https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x={x}&y={y}"
                headers={'Authorization':'KakaoAK {}'.format(self.rest_api_key)}
                res = requests.get(url, headers=headers)
                res = res.json()
                addr=res['documents'][0]['address_name']
                x= res['documents'][0]['x']
                y= res['documents'][0]['y']
                folium.Marker(location=[y,x],tooltip=j["병원명"],icon=folium.Icon(color='red',icon='ok')).add_to(marker_c)
                
                
            except KeyError:
                print("===========================")
                print(j["병원명"])
                print("===========================")
                print(x,y)
                print("===========================")
                print(url)
                print("===========================")
                print(res)
                kerr.append(j)
                count += 1

        print (f'오류횟수{count}')
        print ("오류 목록")
        print (kerr)
        m.save('map.html')
        webbrowser.open('map.html')
            
    def make_arr(self) -> bokeh.io:
        # self.test_list2()
        fin_source_data = self.test_list2()
        source = ColumnDataSource(fin_source_data)
        columns = [ TableColumn( field=col, title=col ) for col in fin_source_data.columns ]
        data_table = DataTable(source=source, columns = columns)
        bokeh.io.save(obj=data_table, filename='table.html')
        webbrowser.open('table.html')


    def total(self) -> pd.DataFrame:
        #self.test_list2()
        self.make_map()
        self.make_arr()
        return self.fin_source_data

    def df_full(self) -> pd.DataFrame:
        pd.set_option('display.max_rows', None)
        return self.fin_source_data

        


        

In [153]:
test = Veterinary()
test_df = test.make_map()
test_df

파일경로 : ./fulldata_02_03_01_P_동물병원.csv
파일 인코딩 방식 : EUC-KR


<ipython-input-152-aa7e0f6fa7d2>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.fin_source_data['구분'] = self.fin_source_data['소재지전체주소'].str.split(' ').str.get(0).copy()
C:\Users\ad\miniconda3\lib\site-packages\pandas\core\series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


---- 오류 주소 내용----
경기도 성남시 분당구 미금로 251, 112,201,202,203호 (금곡동, 청솔마을성원아파트)
---- 오류 주소 내용----
서울특별시 노원구 월계로 333 (월계동)
---- 오류 주소 내용----
서울특별시 노원구 월계동 331-1   번지 신세계이마트 2층
---- 오류 주소 내용----
경기도 용인시 수지구 신봉1로 84, 228,229호 (신봉동, 상록신봉종합상가)
---- 오류 주소 내용----
광주광역시 북구 임동로 6 (임동)
---- 오류 주소 내용----
경기도 고양시 덕양구 화중로 76, 102,103호 (화정동, 대감빌딩)
---- 오류 주소 내용----
경기도 고양시 일산동구 백마로 223, 103~4호 (장항동, 현대 에뜨레보)
---- 오류 주소 내용----
경기도 고양시 일산동구 위시티2로 13, 102호 (식사동)
---- 오류 주소 내용----
충청북도 청주시 상당구 1순환로 1233, 106,107, 108호 (용암동)
---- 오류 주소 내용----
경상남도 진주시 금산면 송백로 46
---- 오류 주소 내용----
전라북도 전주시 덕진구 만성중앙로 25, 미네르바빌딩 1층 101,102,103호 (만성동)
---- 오류 주소 내용----
경기도 평택시 청북읍 안청로2길 33-2, 105,106호
---- 오류 주소 내용----
경기도 성남시 분당구 동판교로52번길 6, 외 1필지(분당구 동판교로52번길 8) 1층 (백현동)
---- 오류 주소 내용----
경기도 성남시 분당구 동판교로177번길 25, 101,102,103호 (삼평동, 판교 호반 써밋 플레이스)
---- 오류 주소 내용----
경기도 성남시 분당구 운중로 135, 101,102호 (운중동, 더원스퀘어)
---- 오류 주소 내용----
경기도 안산시 단원구 중앙대로 899, 1.2층 (고잔동)
---- 오류 주소 내용----
경기도 성남시 분당구 동판교로 226, 204, 205, 206호 (삼평동, 봇들마을4단지아파트)


In [99]:
a = Veterinary()
a1 = a.choose_columns()
a1.info()

파일경로 : ./fulldata_02_03_01_P_동물병원.csv
파일 인코딩 방식 : EUC-KR
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4869 entries, 0 to 4868
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   병원명     4869 non-null   object 
 1   영업상태    4869 non-null   object 
 2   지번      4787 non-null   object 
 3   도로명     4840 non-null   object 
 4   위도      4766 non-null   float64
 5   경도      4766 non-null   float64
 6   지역      4787 non-null   object 
dtypes: float64(2), object(5)
memory usage: 304.3+ KB


<ipython-input-98-d34863c92c92>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fin_source_data['구분'] = fin_source_data['소재지전체주소'].str.split(' ').str.get(0).copy()


In [133]:
test = Veterinary()
test2 = test.make_arr()
test2

파일경로 : ./fulldata_02_03_01_P_동물병원.csv
파일 인코딩 방식 : EUC-KR


<ipython-input-132-f4c360afe1a2>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.fin_source_data['구분'] = self.fin_source_data['소재지전체주소'].str.split(' ').str.get(0).copy()
C:\Users\ad\miniconda3\lib\site-packages\pandas\core\series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


---- 오류 주소 내용----
경기도 성남시 분당구 미금로 251, 112,201,202,203호 (금곡동, 청솔마을성원아파트)
---- 오류 주소 내용----
충청북도 괴산군 불정면 목도리 296-7번지
---- 오류 주소 내용----
충청북도 괴산군 괴산읍 동부리 672-4번지
---- 오류 주소 내용----
서울특별시 노원구 월계로 333 (월계동)
---- 오류 주소 내용----
서울특별시 노원구 월계동 331-1   번지 신세계이마트 2층
---- 오류 주소 내용----
경기도 용인시 수지구 신봉1로 84, 228,229호 (신봉동, 상록신봉종합상가)
---- 오류 주소 내용----
광주광역시 북구 임동로 6 (임동)
---- 오류 주소 내용----
경기도 고양시 덕양구 화중로 76, 102,103호 (화정동, 대감빌딩)
---- 오류 주소 내용----
경기도 고양시 일산동구 백마로 223, 103~4호 (장항동, 현대 에뜨레보)
---- 오류 주소 내용----
경기도 고양시 일산동구 위시티2로 13, 102호 (식사동)
---- 오류 주소 내용----
경기도 고양시 일산동구 식사동 344번지 자이주상복합아파트 상가 B동 211-1호
---- 오류 주소 내용----
경기도 고양시 일산서구 탄현동 1573-5번지 현해프라자 203호
---- 오류 주소 내용----
충청북도 청주시 상당구 1순환로 1233, 106,107, 108호 (용암동)
---- 오류 주소 내용----
경상남도 합천군 가야면 황산리 18-8번지
---- 오류 주소 내용----
경상남도 진주시 금산면 송백로 46
---- 오류 주소 내용----
전라북도 전주시 덕진구 만성중앙로 25, 미네르바빌딩 1층 101,102,103호 (만성동)
---- 오류 주소 내용----
경기도 평택시 청북읍 안청로2길 33-2, 105,106호
---- 오류 주소 내용----
경기도 성남시 분당구 동판교로52번길 6, 외 1필지(분당구 동판교로52번길 8) 1층 (백현동)
---- 오류 주소 내용---

C:\Users\ad\miniconda3\lib\site-packages\bokeh\io\saving.py:126: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
C:\Users\ad\miniconda3\lib\site-packages\bokeh\io\saving.py:139: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")


In [114]:
len(a2)

4869

In [56]:
a = pd.DataFrame(data=[
    [1,2,3,4,5],
    [6,7,8,9,10],
    [11,12,13,14,15],
    [16,17,18,19,20],
    [21,22,23,24,25]],
    columns=list('abcde'))
c = []
while True:
    print(list(a.columns))
    b = input("추가할 컬럼을 선택 // 종료 0")
    if b == '0':
        break
    c.append(b)

print(c)
#cc = a.loc[:,['b','c','d']]
cc = a.loc[:,c]
cc


    



['a', 'b', 'c', 'd', 'e']
[]


""
0
1
2
3
4


In [50]:
a = ['1','2','3']
b = list(map(int, a))
b

[1, 2, 3]

In [52]:
type(b)

list

In [35]:
a = Veterinary()
b = a.find_encoding()
b

파일경로 : ./fulldata_02_03_01_P_동물병원.csv
파일 인코딩 방식 : EUC-KR


,번호,개방서비스명,개방서비스id,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태구분코드,영업상태명,상세영업상태코드,...,데이터갱신구분,데이터갱신일자,업태구분명,좌표정보(x),좌표정보(y),업무구분명,상세업무구분명,권리주체일련번호,총종업원수,Unnamed: 32
0,1,동물병원,02_03_01_P,4440000,444000001020210001,20210202,NaN,1,영업/정상,0000,...,I,2021-02-04 00:23:03.0,NaN,269459.440980,297446.921926,동물병원,NaN,000,NaN,NaN
1,2,동물병원,02_03_01_P,3740000,374000001020210001,20210126,NaN,1,영업/정상,0000,...,U,2021-04-30 02:40:00.0,NaN,196493.936553,418242.406823,동물병원,NaN,000,NaN,NaN
2,3,동물병원,02_03_01_P,5250000,525000001020210001,20210122,NaN,1,영업/정상,0000,...,I,2021-01-24 00:23:04.0,NaN,418279.642270,361268.661686,동물병원,NaN,000,NaN,NaN
3,4,동물병원,02_03_01_P,4690000,469000001020210001,20210122,NaN,1,영업/정상,0000,...,I,2021-01-24 00:23:04.0,NaN,185270.307816,240487.309876,동물병원,NaN,000,NaN,NaN
4,5,동물병원,02_03_01_P,5710000,573000001020200003,20201209,NaN,1,영업/정상,0000,...,I,2020-12-11 00:23:07.0,NaN,238737.923591,352447.666104,동물병원,NaN,000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9176,9177,동물병원,02_03_01_P,3450000,345000001019810005,19810824,NaN,3,폐업,0002,...,I,2018-08-31 23:59:59.0,NaN,344723.182424,266253.208998,동물병원,NaN,000,NaN,NaN
9177,9178,동물병원,02_03_01_P,3620000,362000001020170001,20170105,NaN,3,폐업,0002,...,I,2018-08-31 23:59:59.0,NaN,189145.000000,191466.000000,동물병원,NaN,L00,NaN,NaN
9178,9179,동물병원,02_03_01_P,3620000,362000001019910001,19910710,NaN,3,폐업,0002,...,U,2019-01-24 02:40:00.0,NaN,188395.475651,189857.285188,동물병원,NaN,000,NaN,NaN
9179,9180,동물병원,02_03_01_P,3620000,362000001020160001,20160121,NaN,3,폐업,0002,...,I,2018-08-31 23:59:59.0,NaN,189132.523692,191478.416598,동물병원,NaN,L00,NaN,NaN


In [19]:
# !pip install chardet
import pandas as pd
import chardet

def find_encoding(filename :str) -> str:
    read_file = open(filename, 'rb').read()        # 파일 읽기모드 rb는 read + 바이너리 파일( 바이너리 파일 읽을때 사용 )
    result    = chardet.detect(read_file)
    charenc   = result['encoding']
    return charenc

source_path = "./fulldata_02_03_01_P_동물병원.csv"
# csv 출처(행안부) https://www.data.go.kr/data/15045050/fileData.do

file_encoding = find_encoding(source_path)
open_source = pd.read_csv(source_path, encoding=file_encoding)

print(file_encoding)

EUC-KR
